In [1]:
import numpy as np
import pdfplumber
import os
import re
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.styles import Font, PatternFill, Border, Alignment

## Step 1: Fill Excel "DoesCal.xlsx" (Manual) :
    In Sheet "Initial", fill all cells in yellow with fixed units.

## Step 2: Save PACE4 results in Folder "PACE4" (Manual):
    Make sure the beam energy for PACE4 is same as the one in Excel Sheet "Initial"

## Step 3: Run the code to copy info from PACE4 result to "DoesCal.xlsx"

### Code(Extract_CompoundInf): extract Compound Info from PDF

In [2]:
def Extract_CompoundInf(text):
    # Return Values:
    Coulomb_Barrier = -1
    Center_Mass = -1
    Recoil_Energy = -1
    Beam_Energy = -1
    Compound_A = -1
    
    lines = text.split('\n')
    # loop all lines in this text(page)
    for i, line in enumerate(lines):
        if "Bass fusion xsection for E = " in line:
            match = re.search(r'Bass fusion xsection for E = (\d+(\.\d+)?)\s*MeV', line)
            if match:
                Beam_Energy = float(match.group(1))
                continue
        if "Barrier height" in line:
            match = re.search(r'Barrier height is (\d+(\.\d+)?)\s*MeV', line)
            if match:
                Coulomb_Barrier = float(match.group(1))
                continue
        if "Center of mass energy (MeV)" in line:
            match = re.search(r'Center of mass energy \(MeV\)\s*(\d+(\.\d+)?)', line)
            if match:
                Center_Mass = float(match.group(1))
                continue
        if "Compound nucleus recoil energy (MeV)" in line:
            match = re.search(r'Compound nucleus recoil energy \(MeV\)\s*(\d+(\.\d+)?)', line)
            if match:
                Recoil_Energy = float(match.group(1))
                break
        if "Compound nucleus " in line and Compound_A < 0:
            columns = line.split()
            Compound_A = float(columns[-1])

    if Beam_Energy<0:
        print("Beam energy NOT Found")
    if Coulomb_Barrier<0:
        print("Barrier height NOT Found")
    if Center_Mass<0:
        print("Center of mass energy NOT Found")
    if Recoil_Energy<0:
        print("Compound nucleus recoil energy NOT Found")
    if Compound_A<0:
        print("Compound nucleus info NOT Found")
    
        
    return Beam_Energy, Coulomb_Barrier, Center_Mass, Recoil_Energy, Compound_A

### Code(Extract_ProductInf): extract Production Info from PDF

In [3]:
def Extract_ProductInf(pdf):
    start_flag = False 
    end_flag = False
    Production_Inf = []
    for page_num, page in enumerate(pdf.pages[1:]):
        if start_flag and end_flag: break
        text = page.extract_text()
        if text:
            lines = text.split('\n')
            for i, line in enumerate(lines):
                if "Yields of residual nuclei" in line: 
                    start_flag = True
                    continue
                if "Angular distribution results" in line: 
                    end_flag = True
                    break
                if start_flag and not end_flag:
                    Production_Inf.append(line.split())
    if not Production_Inf:
        print("Yields of residual nuclei NOT Found")
    return Production_Inf

### Code(Filter_Inf): filter Production Info

In [4]:
def Filter_Inf(Production):
    # If production cross pages, there will be line written in "Production" with one element which returns page number
    # So we need to delete it first
    Production = [row for row in Production if len(row) > 1]

    # Take out the last line which contains total information for the reaction
    # Then delete it from "Production"
    Total = Production[-1]
    Production = Production[:-1]

    # Add "isotop for the Column name"
    Production[0].insert(3, "isotope")
    
    # Only Record Z, N, A, isotope, xsection information into Excel
    # And convert all values to int or float
    Production_Record = [[row[0], row[1], row[2], row[3], row[6]] for row in Production]
    for row in Production_Record[1:]:
        row[0] = int(row[0])  # Convert Z to int
        row[1] = int(row[1])  # Convert N to int
        row[2] = int(row[2])  # Convert A to int
        row[4] = float(row[4])  # Convert x-section(mb) to float
    
    return Total, Production_Record

### Code(Write_Excel): write info to excel

In [5]:
def Write_Excel(Production_Record, Total, file_path, Beam_Energy=0, Recoil_Energy=0, Compound_A = 0):
    #================== Update "Initial" sheet ================#
    # Load the existing workbook and the "Initial" sheet
    workbook = load_workbook(file_path)
    existing_df = pd.read_excel(file_path, sheet_name="Initial", engine='openpyxl')
    
    # Update the existing DataFrame with recoil energy and total x-sec for the current beam energy
    row_index = existing_df[existing_df["Beam Energy (MeV)"] == Beam_Energy].index[0]
    existing_df.at[row_index, "Recoil Energy (MeV)"] = Recoil_Energy
    existing_df.at[row_index, "Total xsec (mb)"] = float(Total[-1])
    thickness = existing_df.at[row_index,"Target layer (um)"]

    # Write the updated DataFrame back to the "Initial" sheet
    existing_worksheet = workbook["Initial"] 
    for row in range(len(existing_df)):
        for col in range(len(existing_df.columns)):
            existing_worksheet.cell(row=row + 2, column=col + 1, value=existing_df.iat[row, col])  # +2 to account for header
    # Save the workbook after updating the existing sheet
    workbook.save(file_path)

    #================== Create new sheet for PACE4 result ================#
    # Convert the first row as headers and the rest recorded into DataFrame
    df = pd.DataFrame(Production_Record[1:], columns=Production_Record[0])
    
    # Create a new row for the DataFrame based on Total
    total_row = pd.DataFrame({
        'Z': [Total[0]],  # The first element of Total in 'Z' column
        'x-section(mb)': [float(Total[-1])]  # The last element of Total in 'x-section(mb)' column
    })
    
    # Append the total_row to the DataFrame
    df = pd.concat([df, total_row], ignore_index=True)

    # Insert a new column "Recoil Energy(MeV)" after "x-section(mb)"
    df.insert(df.columns.get_loc('x-section(mb)') + 1, 'Recoil_E(MeV)', 
              df['A'].apply(lambda A: Recoil_Energy / Compound_A * A))
    
    # Create a new sheet name based on Beam Energy
    new_sheet_name = f'{int(thickness)}um_{int(Beam_Energy)}MeV'

    # Now create the new sheet with the updated DataFrame
    with pd.ExcelWriter(file_path, mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
        df.to_excel(writer, sheet_name=new_sheet_name, index=False)

    # Reopen the workbook to apply formatting
    workbook = load_workbook(file_path)

    # Set column width and alignment for the new sheet
    worksheet = workbook[new_sheet_name]
    for column in worksheet.columns:
        column_letter = column[0].column_letter
        worksheet.column_dimensions[column_letter].width = 16
        for cell in column:  # Align each cell in the column
            cell.alignment = Alignment(horizontal='right')
   
    # Final save
    workbook.save(file_path)

### Code(RWProcess): Read PDF and Write EXCEL

In [6]:
def RWProcess(pdf_path, excel_path):
    with pdfplumber.open(pdf_path) as pdf:
        # Extract basic information for this reaction
        page = pdf.pages[0]
        text = page.extract_text()
        Beam_Energy, Coulomb_Barrier, Center_Mass, Recoil_Energy, Compound_A = Extract_CompoundInf(text)
        # Extract info for productions from the reaction
        Production = Extract_ProductInf(pdf)

    # Filter Production info to write them into Spreadsheet
    Total, Production_Record = Filter_Inf(Production)

    # Write Production Info into Excel:
    Write_Excel(Production_Record, Total, excel_path, Beam_Energy, Recoil_Energy, Compound_A)    

### Code(RWPACE4): loop PACE4 folder

In [7]:
def RWPACE4(folder_path, excel_path):
    for pdf_file in os.listdir(folder_path):
        if pdf_file.endswith(".pdf"):
            pdf_path = os.path.join(folder_path, pdf_file)
            RWProcess(pdf_path, excel_path)

## Step 4: Extract Half-Lives
### 4.1 Read half-life information from NNDC :
1. Go to Nuclear Levels and Gammas Search;
2. Give the range for Z or A or N to search the result (I usually set half-live range > 1 sec);
3. Save the result page as PDF (Right click -> choose print -> Save as PDF);

* No space in any path

### 4.2 Extract half-lives:
1. Only write the half-life info into the .txt file same name as .pdf file (ONE TIME Operation!!!! Unless you want to update the old one)
2. Extract the half-lives from .txt file, which is faster than reading .pdf file;
3. Add half-life info to isotopes saved in the Excel sheet created in Step 3;

### Code(Convert_PDF_to_TXT): convert t1/2 PDF to TXT

In [1]:
def Convert_PDF_to_TXT(pdf_path, flag_overwrite = False):
    # Get the base name of the PDF file (without the extension) using string operations
    base_name = pdf_path.split('/')[-1].replace('.pdf', '')
    # Create the output TXT file name
    txt_path = pdf_path.replace('.pdf', '.txt')
    
    # if txt file exists and don't need to be replaced, return empty
    if os.path.exists(txt_path) and flag_overwrite:
        return
    # if txt file doesn't exist or you want to overwrite it
    else:
        # Open the PDF file
        with pdfplumber.open(pdf_path) as pdf:
            text_to_write = []
            capture_text = False # will be "True" when detect "Nucleus E level(keV) Jπ T 1/2" where start extracting info
            # Iterate through each page
            for page in pdf.pages:
                # Extract the text from the page
                text = page.extract_text() 
                # Split the text into lines
                lines = text.split('\n')
                for line in lines:
                    # Skip the unwanted header and footer lines
                    #if "page" in line.lower() and "," in line and ("AM" in line or "PM" in line):
                    if "AM" in line or "PM" in line:
                        continue
                    if "https://www.nndc.bnl.gov" in line:
                        continue     
                    # Start capturing text after the target start line
                    if "Nucleus E level(keV) Jπ T 1/2" in line:
                        capture_text = True
                        continue
                    # Stop capturing text after the target end line
                    if "Gamma Information" in line:
                        capture_text = False
                        continue
                    if "≥" in line or ">" in line:
                        continue
                    if "eV" in line:
                        continue
                    # Capture the text
                    if capture_text:
                        text_to_write.append(line)
        # Write the captured text to the dynamically named txt file
        with open(txt_path, "w") as output_file:
            output_file.write("\n".join(text_to_write))

### mini Code(parse_t_half): parse t1/2 info in txt file

In [9]:
# ============= Function to parse the t_half value into t_value + t_unit ==============#
# the input "t_half" should have one character (no-digit) part
def parse_t_half(t_half):
    # if t1/2 = stable, will return the age of the universe
    if t_half.upper() == "STABLE":
        return 1.4e10, "y"
    matches = re.findall(r'(\d+(?:\.\d+)?(?:e[-+]?\d+)?)\s*([a-zA-Z]+)', t_half)
    # Initialize variables for the extracted values
    t_value = -1
    t_unit = None
    # If matches are found, assign the number and unit
    if matches:
        t_value = float(matches[0][0])  # First number found
        t_unit = matches[0][1]    # Corresponding unit
    return t_value, t_unit

### Code(Extract_HalfLife): Extract A, nucleus_name, t1/2_value and t1/2_unit into an array

In [10]:
# ============= Function to extract t1/2 from txt ==============#
# extracted_data = [A,"nucleus_name", t1/2, "unit"]
def Extract_HalfLife(txt_path):
    # Open the existing TXT file and process it
    with open(txt_path, "r") as input_file:
        extracted_data = []
        # Skip the first line (header)
        next(input_file)
        for line in input_file:
            # Skip any empty lines
            if not line.strip():
                continue
            # Split the line into columns
            columns = line.split()
            # Extract Nucleus (separated into number and letters, like "1H" to "1" and "H")
            nucleus_full = columns[0]
            nucleus_number = ''.join([char for char in nucleus_full if char.isdigit()])
            nucleus_letters = ''.join([char for char in nucleus_full if char.isalpha()])
            # Extract and parse T 1/2
            t_half = ' '.join(columns[3:])
            value, unit = parse_t_half(t_half)
            if value > 0:
                # Append the extracted data as a tuple
                extracted_data.append((float(nucleus_number), nucleus_letters, value, unit))
        return extracted_data

### mini Code(HalfLife_Unit_Factor): scaling factor to convert t1/2 in seconds

In [11]:
#============= convert all half-lives to seconds =============#
def HalfLife_Unit_Factor(unit):
    fac = 0
    if unit == "s":
        fac = 1;
    if unit == "m":
        fac = 60;
    if unit == "h":
        fac = 60*60;
    if unit == "d":
        fac = 24*60*60;
    if unit == "y":
        fac = 365*24*60*60;
    return fac

### mini Code(Fill_HalfLife_Sheet): Fill t1/2 info into one Sheet

In [12]:
#============= Fill Half-Life to one sheet in Excel =============#
def Fill_HalfLife_Sheet(txt_path, excel_path, sheet_name):
    sheets = pd.read_excel(excel_path, sheet_name=None)  # Load all sheets
    df = sheets[sheet_name]
    # Add new columns for "t1/2" and "unit" if they don't already exist
    if "t1/2" not in df.columns:
        df["t1/2"] = -1.0
    df.at[df.index[-1], "t1/2"] = np.nan
    if "unit" not in df.columns:
        df["unit"] = ""
    if "t1/2(s)" not in df.columns:
        df["t1/2(s)"] = -1.0
    df.at[df.index[-1], "t1/2(s)"] = np.nan
    # Iterate over each entry in the extracted_data array

    extracted_data = Extract_HalfLife(txt_path);
    for data in extracted_data:
        A = data[0]       # A
        isotope = data[1] # "Isotope"
        t_value = data[2] # t1/2
        t_unit = data[3]  # "unit"
        # Loop over all rows in the DataFrame to check for matches
        for idx in range(len(df)-1):
            if df.at[idx, "A"] == A and df.at[idx, "isotope"].lower() == isotope.lower():
                # Fill the first matching row with t1/2 and unit data
                if df.at[idx, "t1/2"] < 0:
                    df.at[idx, "t1/2"] = t_value
                    df.at[idx, "unit"] = t_unit
                    df.at[idx, "t1/2(s)"] = t_value * HalfLife_Unit_Factor(t_unit)
                    break
                else:
                    # If there are already values, create a new row
                    new_row = df.loc[idx].copy()
                    new_row["t1/2"] = t_value
                    new_row["unit"] = t_unit
                    new_row["t1/2(s)"] = t_value * HalfLife_Unit_Factor(t_unit)
                    # Append " isomer" to the Isotope in the new row
                    new_row["isotope"] = f"{new_row['isotope']} isomer"
                    df = pd.concat([df.iloc[:idx + 1], pd.DataFrame([new_row]), df.iloc[idx + 1:]]).reset_index(drop=True)
                    break
    # Save df back to Excel
    with pd.ExcelWriter(excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
        df.to_excel(writer, sheet_name=sheet_name, index=False)
    # Reopen the workbook to apply formatting
    workbook = load_workbook(excel_path)
    # Set column width and alignment for the new sheet
    worksheet = workbook[sheet_name]
    for column in worksheet.columns:
        column_letter = column[0].column_letter
        worksheet.column_dimensions[column_letter].width = 16
        for cell in column:  # Align each cell in the column
            cell.alignment = Alignment(horizontal='right')
    # Save the workbook
    workbook.save(excel_path)

### Code(Fill_HalfLife_Exce): Fill t1/2 info to all sheets in Exxcel

In [13]:
#========================== Fill Half-Life to all sheets ("MeV") in Excel =============================#
def Fill_HalfLife_Excel(txt_path, excel_path):
    # To check if txt file exists
    if os.path.exists(txt_path) is False:
        # Get the base name of the txt file (without the extension) using string operations
        base_name = txt_path.split('/')[-1].replace('.txt', '')
        # Create the output TXT file name
        pdf_path = txt_path.replace('.txt', '.pdf')
        Convert_PDF_to_TXT(pdf_path)
    workbook = load_workbook(excel_path)
    sheet_names = workbook.sheetnames   
    # Iterate over all sheets and call Fill_HalfLife_Sheet() if the sheet name includes "MeV"
    for sheet_name in sheet_names:
        if "MeV" in sheet_name:
            Fill_HalfLife_Sheet(txt_path, excel_path, sheet_name)

## Step 5: Calculate Production Rate
---
1. Read information from Sheet "Initial"
2. Calculate production rates in each sub Sheet

* Be really really careful on unit, the current code is based on unit:
  1. thickness: mg/cm2;
  2. molar mass: g;
  3. x-section: mb;
  4. beam rate: pps;

### Code(ReadInitial): Read Sheet "Initial":
1. thickness in "mg/cm2";
2. molar mass in "g"
3. beam rate in "pps"

In [14]:
def ReadInitial(excel_path):
    sheets = pd.read_excel(excel_path, sheet_name=None)  # Load all sheets
    df = sheets["Initial"]
    
    # locate the target info
    target_col_index = df.columns.get_loc('Target')
    thick_row_idx = df[df.iloc[:, target_col_index + 2] == 'mg/cm2'].index[0]
    thick_tot = df.iloc[thick_row_idx, target_col_index + 1]
    mass_row_idx = df[df.iloc[:, target_col_index + 2] == 'g'].index[0]
    mass = df.iloc[mass_row_idx, target_col_index + 1]

    # locate the beam info
    beam_col_index = df.columns.get_loc('Beam')
    rate_row_idx = df[df.iloc[:, beam_col_index + 2] == 'pps'].index[0]
    rate = df.iloc[rate_row_idx, beam_col_index + 1]

    return thick_tot, mass, rate

### Code(Cal_Prodx_Rate): Calculate production rate in on Sheet

In [15]:
def Cal_Prodx_Rate(df, NLayer, thick_tot, mass, rate):
    if "Prodx_Rate(pps)" not in df.columns:
        df["Prodx_Rate(pps)"] = -1.0
    df.at[df.index[-1], "Prodx_Rate(pps)"] = np.nan
    for idx in range(len(df)-1):
        xsec = df.at[idx,'x-section(mb)']
        # Eqn: prodx_rate = xsec(m2) * unit_thick(g/m2) / mass(g) *rate (atom/s)
        # 1 mb = 1e-3 barn = 1e-3 * 1e-28 m2
        # 1 mg/cm2 = 10 g/m2
        df.at[idx,'Prodx_Rate(pps)'] = (xsec/1e3/1e28)*((thick_tot/NLayer)*10)/mass*rate
    return df

### Code(Write_Prodx_Rate): Update all sheets in Excel

In [16]:
def Write_Prodx_Rate(excel_path, NLayer):
    thick_tot, mass, rate = ReadInitial(excel_path)

    sheets = pd.read_excel(excel_path, sheet_name=None)
    workbook = load_workbook(excel_path)
    sheet_names = workbook.sheetnames   
    # Iterate over all sheets and call Fill_HalfLife_Sheet() if the sheet name includes "MeV"
    for sheet_name in sheet_names:
        if "MeV" in sheet_name:
            df = sheets[sheet_name]
            df = Cal_Prodx_Rate(df, NLayer, thick_tot, mass, rate)
            # Save df back to Excel
            with pd.ExcelWriter(excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
                df.to_excel(writer, sheet_name=sheet_name, index=False)
            # Reopen the workbook to apply formatting
            workbook = load_workbook(excel_path)
            # Set column width and alignment for the new sheet
            worksheet = workbook[sheet_name]
            for column in worksheet.columns:
                column_letter = column[0].column_letter
                worksheet.column_dimensions[column_letter].width = 16
                for cell in column:  # Align each cell in the column
                    cell.alignment = Alignment(horizontal='right')
            # Save the workbook
            workbook.save(excel_path)

## Step 6: Generate Stopping Ratio
---
1. Currently, set stopping ratios for all isotopes under all cases are 1, which means all productions will stop in the target;
2. Generate values and save into Excel and create txt foils (optional, named with sub sheet name):

+ ToDo:
Either SRIM results or GEANT4 codes for thses values, based on recoil energy and the rest of thickness

### Code(Generate_Stop_Ratio):
+ Default:
+ Generate stopping ratio of all isotopes under all cases = 1;</br>
+ Only update Excel

In [17]:
def Generate_Stop_Ratio(excel_path):
    sheets = pd.read_excel(excel_path, sheet_name=None)
    workbook = load_workbook(excel_path)
    sheet_names = workbook.sheetnames
    for sheet_name in sheet_names:
        if "MeV" in sheet_name:
            df = sheets[sheet_name]
            if "Stop_Ratio" not in df.columns:
                df["Stop_Ratio"] = 1
            else: 
                print("Stopping Ratio has been generated")
                return 
            df.at[df.index[-1], "Stop_Ratio"] = np.nan
            with pd.ExcelWriter(excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
                df.to_excel(writer, sheet_name=sheet_name, index=False)
            # Reopen the workbook to apply formatting
            workbook = load_workbook(excel_path)
            # Set column width and alignment for the new sheet
            worksheet = workbook[sheet_name]
            for column in worksheet.columns:
                column_letter = column[0].column_letter
                worksheet.column_dimensions[column_letter].width = 16
            for cell in column:  # Align each cell in the column
                cell.alignment = Alignment(horizontal='right')
            # Save the workbook
            workbook.save(excel_path)

## Step 7: Calculate Production A(Bq) and Summarize Sub Sheet

### 7.1: Calculate Production A(Bq):
A = Prodx_Rate * (1-exp(-0.693/halflife * beam_time)) * Stop_Ratio
+ Everything in the equation above is in seconds

### Code(Cal_Prodx_A)

In [18]:
def Cal_Prodx_A(excel_path):
    sheets = pd.read_excel(excel_path, sheet_name=None)
    workbook = load_workbook(excel_path)
    sheet_names = workbook.sheetnames

    df = sheets["Initial"]
    beam_col_index = df.columns.get_loc('Beam')
    run_time_row_idx = df[df.iloc[:, beam_col_index + 2] == 'sec'].index[0]
    run_time = df.iloc[run_time_row_idx, beam_col_index + 1]

    for sheet_name in sheet_names:
        if "MeV" in sheet_name:
            df = sheets[sheet_name]
            if "Prodx_A(Bq)" not in df.columns:
                df["Prodx_A(Bq)"] = -1.0
            df.at[df.index[-1], "Prodx_A(Bq)"] = np.nan
            for idx in range(len(df)-1):
                prodx_rate = df.at[idx,'Prodx_Rate(pps)']
                halflife = df.at[idx,'t1/2(s)']
                stop_ratio = df.at[idx,'Stop_Ratio']
                df.at[idx,"Prodx_A(Bq)"] = prodx_rate * (1 - np.exp(-0.693/halflife * run_time)) * stop_ratio


            # Save df back to Excel
            with pd.ExcelWriter(excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
                df.to_excel(writer, sheet_name=sheet_name, index=False)
            # Reopen the workbook to apply formatting
            workbook = load_workbook(excel_path)
            # Set column width and alignment for the new sheet
            worksheet = workbook[sheet_name]
            for column in worksheet.columns:
                column_letter = column[0].column_letter
                worksheet.column_dimensions[column_letter].width = 16
                for cell in column:  # Align each cell in the column
                    cell.alignment = Alignment(horizontal='right')
            # Save the workbook
            workbook.save(excel_path)

### 7.2 Summarize all sub sheets (including A, isotope, A(Bq) @ *um)
1. Summarize all production activities in one sheet;
2. Calculate sum activity of each production at cool down time = 0s;
3. (optional) Delete production row if A = 0;

### Code(summarize)

In [27]:
def summarize(excel_path, clean_table = True):
    sheets = pd.read_excel(excel_path, sheet_name=None)
    workbook = load_workbook(excel_path)

    # Extract data from sheets named "*um_*MeV"
    data_frames = []
    for sheet_name in workbook.sheetnames:
        if 'um_' in sheet_name and 'MeV' in sheet_name:
            df = sheets[sheet_name][['A', 'isotope', 'Prodx_A(Bq)']]
            df.columns = ['A', 'isotope', f'A(Bq)@{sheet_name.split("um_")[0]}um']
            data_frames.append((int(sheet_name.split('um_')[0]), df))

    # Sort data_frames by the numeric value in the sheet names
    data_frames.sort(key=lambda x: x[0])

    # Extract only the DataFrames in the correct order
    data_frames = [df for _, df in data_frames]

    # Merge all data frames on 'A' and 'isotope'
    merged_df = data_frames[0]
    for df in data_frames[1:]:
        merged_df = pd.merge(merged_df, df, on=['A', 'isotope'], how='outer')

    # Reorder by "A" from large to small, and then by "isotope" keyword
    merged_df['isotope_keyword'] = merged_df['isotope'].str.replace(' isomer', '')
    merged_df = merged_df.sort_values(by=['A', 'isotope_keyword', 'isotope'], ascending=[False, True, True])
    # Drop the temporary column used for sorting
    merged_df = merged_df.drop(columns=['isotope_keyword'])
    
    # Round all "Prodx" columns to 2 decimal places
    for col in merged_df.columns:
        if "um" in col:
            merged_df[col] = merged_df[col].fillna(0).round(2)

    # Add the new column "A(Bq)@cd = 0s" if it doesn't exist
    if "A(Bq)@cd = 0s" not in merged_df.columns:
        merged_df["A(Bq)@cd = 0s"] = -1

    # Calculate the sum of all "A(Bq)@*um" columns for each row
    sum_columns = [col for col in merged_df.columns if "um" in col]
    merged_df["A(Bq)@cd = 0s"] = merged_df[sum_columns].sum(axis=1)

    # Remove rows where "A(Bq)@cd = 0s" is 0
    if clean_table:
        merged_df = merged_df[merged_df["A(Bq)@cd = 0s"] != 0]
    
    # Save the updated workbook
    with pd.ExcelWriter(excel_path, engine='openpyxl', mode='a', if_sheet_exists = 'replace') as writer:
        merged_df.to_excel(writer, sheet_name='Summary_A', index=False)

    # Reopen the workbook to apply formatting
    workbook = load_workbook(excel_path)
    # Set column width and alignment for the new sheet
    worksheet = workbook['Summary_A']
    for column in worksheet.columns:
        column_letter = column[0].column_letter
        worksheet.column_dimensions[column_letter].width = 14
        for cell in column:  # Align each cell in the column
            cell.alignment = Alignment(horizontal='right')
            # Save the workbook
    workbook.save(excel_path)

## Test!!!

In [28]:
excel_path = "DoesCal.xlsx" # Initial Fill after Step 1
pace4_path = "PACE4/" # It is a folder containing all PDF of PACE4 results after Step 2

# Update PACE4 results to Excels (Codes in Step 3)
RWPACE4(pace4_path, excel_path)

t_path = "HalfLife/Levels_Results.txt" 
# Fill t1/2 info in to Excel (Codes in Step 4.2)
Fill_HalfLife_Excel(t_path, excel_path)

# Calculate production rates in sheet "..MeV"
# In this example, 15um target foil is separated by 5 layers. Each layer is 3um;
NLayer = 5
Write_Prodx_Rate(excel_path, NLayer) 

# Generate Stopping Ratio (Step 6)
Generate_Stop_Ratio(excel_path)

# Calculate Production Acitivity at cd = 0 s (Step 7.1)
Cal_Prodx_A(excel_path)

# Summarize Production info so far to a new sheet "Summary" (Step 7.2)
summarize(excel_path)